In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
calendar.head(5)

In [ ]:
calendar["events_names"] = calendar[["event_name_1", "event_name_2"]].fillna('').sum(axis=1).replace('', 'None')
calendar.events_names

In [ ]:
calendar['events_types'] = calendar[['event_type_1', 'event_type_2']].fillna('').sum(axis=1).replace('', 'None')

In [ ]:
calendar['next_events_names'] = calendar[['event_name_1', 'event_name_2']].fillna('').sum(axis=1).replace('', np.nan).fillna(method='bfill')
calendar['next_events_names'] 

In [ ]:
calendar['next_events_types'] =calendar[['event_type_1', 'event_type_2']].fillna('').sum(axis=1).replace('', np.nan).fillna(method='bfill')

calendar['next_events_types']

In [ ]:
calendar['event_name_1'] = pd.Categorical(calendar['event_name_1'].fillna('None'))
calendar['event_type_1'] = pd.Categorical(calendar['event_type_1'].fillna('None'))
calendar['event_name_2'] = pd.Categorical(calendar['event_name_2'].fillna('None'))
calendar['event_type_2'] = pd.Categorical(calendar['event_type_2'].fillna('None'))
calendar.head(5)

In [ ]:
calendar['d'] = pd.Categorical(calendar['d'])

In [ ]:
calendar['wm_yr_wk'] = pd.Categorical(calendar['wm_yr_wk'],categories=calendar.wm_yr_wk.unique(), ordered=True)

In [ ]:
calendar['date'] = pd.to_datetime(calendar['date'], format='%Y-%m-%d')
calendar['wday'] = calendar['wday'].astype(np.int8)

In [ ]:
calendar['snap_CA'] = calendar['snap_CA'].astype(bool)
calendar['snap_TX'] = calendar['snap_TX'].astype(bool)
calendar['snap_WI'] = calendar['snap_WI'].astype(bool)

In [ ]:
calendar.drop(['year', 'weekday', 'month', 'wday'], axis=1, inplace=True)

In [ ]:
calendar['day_of_month'] =calendar['date'].dt.day.astype(np.int8)
calendar['day_of_week'] = calendar['date'].dt.dayofweek.astype(np.int8)

In [ ]:
events_all_datas = calendar[(calendar["event_name_1"] != 'None') | (calendar["event_name_2"] != 'None')]

In [ ]:
prev = 0
for event in events_all_datas.iterrows():
    calendar.loc[prev:event[0], 'days_to_event'] = (calendar.loc[event[0], 'date'] - calendar.loc[prev:event[0], 'date']).dt.days
    prev = event[0]

In [ ]:
calendar['days_to_event'] = calendar['days_to_event'].replace(0, 25).astype(np.int8)

In [ ]:
events_names_dict = calendar['events_names'].value_counts().rank(method='first').to_dict()
events_types_dict = calendar['events_types'].value_counts().rank(method='first').to_dict()
calendar['events_names'] = calendar['events_names'].map(events_names_dict).astype(np.int8)
calendar['events_types'] = calendar['events_types'].map(events_types_dict).astype(np.int8)
calendar['next_events_names'] = calendar['next_events_names'].map(events_names_dict).astype(np.int8)
calendar['next_events_types'] = calendar['next_events_types'].map(events_types_dict).astype(np.int8)

In [ ]:
calendar.drop(['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'], axis=1, inplace=True)

In [ ]:
sell_prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")
sell_prices.head(10)

In [ ]:
sell_prices['store_id'] = pd.Categorical(sell_prices['store_id'])
sell_prices['item_id'] = pd.Categorical(sell_prices['item_id'])
sell_prices['wm_yr_wk'] = pd.Categorical(sell_prices['wm_yr_wk'],categories=sell_prices.wm_yr_wk.unique(), ordered=True)

In [ ]:
sell_prices['dept_id'] = pd.Categorical(sell_prices['item_id'].str.slice(stop=-4))
sell_prices['cat_id'] = pd.Categorical(sell_prices['item_id'].str.slice(stop=-6))
sell_prices['state_id'] = pd.Categorical(sell_prices['store_id'].str.slice(stop=-2))
sell_prices['sell_price'] = sell_prices['sell_price'].replace((np.inf, -np.inf, np.nan), 0).astype(np.float16)

In [ ]:
import category_encoders
from scipy import stats

In [ ]:
VAL_CUTOFF = '2015-05-01'
TRAIN_CUTOFF = '2012-01-01'

In [ ]:
import gc

In [ ]:
sales = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')

In [ ]:
def merge (sell_prices: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    print("merge begin")
    
    sell_prices = sell_prices.merge(calendar, on=['wm_yr_wk'], how='left')
    sell_prices.drop(['wm_yr_wk'], axis=1, inplace=True)
    sell_prices =  sell_prices.merge(pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='demand')[['item_id', 'store_id', 'd', 'demand']], 
              on=['item_id', 'store_id', 'd'],how='left')
    gc.collect()
    
    print("merge end")
    return sell_prices

In [ ]:
def data_type_processing (sell_prices: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    print("data_type_processing start")
    
    sell_prices['store_id'] = pd.Categorical(sell_prices['store_id'])
    sell_prices['item_id'] = pd.Categorical(sell_prices['item_id'])
    sell_prices['id'] = pd.Categorical(sell_prices['item_id'].str.cat(sell_prices['store_id'], sep='_'))
    sell_prices['d'] = sell_prices['d'].str.slice(start = 2).astype(np.int16)
    sell_prices['demand'] = sell_prices['demand'].astype(np.float16)
    
    for i in sell_prices.state_id.unique():
        flag = sell_prices['state_id'] == i
        sell_prices.loc[flag, 'is_snap_avaliable'] = sell_prices.loc[flag, f'snap_{i}']
    
    sell_prices['is_snap_avaliable'] = sell_prices['is_snap_avaliable'].astype(bool)
    sell_prices.drop(['snap_TX', 'snap_CA', 'snap_WI'], axis=1, inplace=True)
    
    gc.collect()
    print("data_type_processing end")
    return sell_prices

In [ ]:
def demand_features (sell_prices: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    print("demand_features start")
    
    days = 29
    sell_prices[f'cumulative_mean_demand_{days}d_ago'] = (sell_prices.groupby(['store_id', 'item_id']).demand
                                                         .transform(lambda x : x.shift(days).expanding().mean())
                                                         .replace((np.inf, -np.inf, np.nan), 0)
                                                         .astype(np.int16))   
    sell_prices[f'cumulative_md_low_demand_{days}d_ago'] = (sell_prices.groupby(['store_id', 'item_id']).demand
                                                         .transform(lambda x : x.shift(days).expanding().median())
                                                         .replace((np.inf, -np.inf, np.nan), 0)
                                                         .astype(np.int16)) 
    for i in [1, 4, 8]:
        sell_prices[f'ewm_mean_{i}w_demand_{days}d_ago'] = (sell_prices.groupby(['store_id', 'item_id']).demand
                                                         .transform(lambda x : x.shift(days).ewm(span=7*i).mean())
                                                         .replace((np.inf, -np.inf, np.nan), 0)
                                                         .astype(np.float16))
        sell_prices[f'rolling_mean_{i}w_demand_{days}d_ago'] = (sell_prices.groupby(['store_id', 'item_id']).demand
                                                         .transform(lambda x : x.shift(days).rolling(7*i).mean())
                                                         .replace((np.inf, -np.inf, np.nan), 0)
                                                         .astype(np.float16))
    gc.collect()
    print("demand_features end")
    return sell_prices

In [ ]:
from sklearn import preprocessing, metrics

In [ ]:
# for lgbm
def encoding (sell_prices: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    print("encoding start")
    
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    
    for features in cat:
        encoder = preprocessing.LabelEncoder() # 将类别编码转为数值编码
        sell_prices[features] = encoder.fit_transform(sell_prices[features]).astype(np.float16)
        del encoder
    
    sell_prices['expected_item_revenue'] = (sell_prices['item_id'] * sell_prices['sell_price']).replace((np.inf, -np.inf, np.nan), 0).astype(np.float16)
    
    print("encoding end")
    return sell_prices

In [ ]:
def cut (sell_prices: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    print("cut start")
    
    sell_prices = sell_prices.loc[sell_prices.date >= TRAIN_CUTOFF]
    
    gc.collect()
    print("cut end")
    return sell_prices.reset_index(drop=True) 

In [ ]:
def train_test_split(sell_prices: pd.core.frame.DataFrame) -> list:
    print("train_test_split start")
    
    X_train = sell_prices[(sell_prices.date < VAL_CUTOFF)].drop(['demand', 'date', 'id', 'd'], axis=1)
    X_valid = sell_prices[(sell_prices['d'] < 1942) & (sell_prices.date >= VAL_CUTOFF)].drop(['demand', 'date', 'id', 'd'], axis=1)
    X_test = sell_prices[sell_prices['d'] >= 1914].drop(['demand', 'd'], axis=1)
    y_train = sell_prices[(sell_prices.date < VAL_CUTOFF)]['demand'].astype(np.int16)
    y_valid = sell_prices[(sell_prices['d'] < 1942) & (sell_prices.date >= VAL_CUTOFF)]['demand'].astype(np.int16)
    
    gc.collect()
    print("train_test_split end")
    return [X_train, X_valid, X_test, y_train, y_valid]

In [ ]:
X_train, X_valid, X_test, y_train, y_valid = (sell_prices
                                              .pipe(merge)
                                              .pipe(data_type_processing)
                                              .pipe(demand_features)
                                              .pipe(encoding)
                                              .pipe(cut)
                                              .pipe(train_test_split))

In [ ]:
X_train=pd.read_csv('../input/dataset-split/X_train.csv')

In [ ]:
X_valid=pd.read_csv('../input/dataset-split/X_valid.csv')
X_test=pd.read_csv('../input/dataset-split/X_test.csv')
y_train=pd.read_csv('../input/dataset-split/y_train.csv')
y_valid=pd.read_csv('../input/dataset-split/y_valid.csv')

In [ ]:
del sell_prices, calendar, sales
gc.collect()

In [ ]:
import lightgbm as lgb

In [ ]:
train_set = lgb.Dataset(X_train, y_train) # 构建lgb的数据集
val_set = lgb.Dataset(X_valid, y_valid)
del X_train, y_train


model = lgb.train(metric='rmse',objective='regression', seed=341,learning_rate=0.07,
        lambda_l1=0.1,num_leaves=64,max_depth=5,colsample_bytree=0.75, bagging_fraction=0.75,
        bagging_freq=1,train_set=train_set, 
        num_boost_round = 2500, 
        early_stopping_rounds = 50, 
        valid_sets = [val_set])
val_pred = model.predict(x_val)
x_val['demand'] = val_pred
print(f'Our val rmse score is {val_score}')

In [ ]:
y_pred = model.predict(X_test) # 预测测试集
X_test['demand'] = y_pred
print('model training costs %7.2f seconds'%(time.time() - start))
gc.collect()

In [ ]:
del model
gc.collect()

In [ ]:
submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")

In [ ]:
predictions = X_test.pivot_table(index='id', columns='date', values='demand').reset_index()
del X_test
gc.collect()

In [ ]:
def get_sub(predictions, submission):
    
    new_cols = [f'F{i}' for i in range(1, 29)]
    validation = predictions.copy()
    validation = validation[validation.columns[1:29]]
    validation = validation.rename({k: v for k, v in zip(validation.columns, new_cols)}, axis=1)
    validation['id'] = predictions['id'].apply(lambda x: x + '_validation')
    
    evaluation = predictions.copy()
    evaluation = evaluation[evaluation.columns[29:]]
    evaluation = evaluation.rename({k: v for k, v in zip(evaluation.columns, new_cols)}, axis=1)
    evaluation['id'] = predictions['id'].apply(lambda x: x + '_evaluation')
    
    sub_1 = pd.merge(submission.loc[:30489, 'id'], validation, how='left')
    sub_2 = pd.merge(submission.loc[30490:, 'id'], evaluation, how='left')
    
    return pd.concat([sub_1, sub_2], axis=0)

In [ ]:
#submission = get_sub(predictions, submission)

In [ ]:
#submission.to_csv('submission.csv', index=False)